In [1]:
import dask.dataframe as dd
from datetime import datetime
import numpy as np
from tqdm import tqdm

In [5]:
df_treino = dd.read_csv("data/train.csv")

In [8]:
df_treino["ts_datetime"] = df_treino.ts_listen.map(lambda x: datetime.fromtimestamp(x))
df_new = df_treino.compute()

In [13]:
users = df_new["user_id"].unique()

In [14]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

def time_offset_rolling_df_count(data_df_ser, window_i_s, min_periods_i=1, center_b=False):
    def calculate_count_ts(ts):
        """Function (closure) to apply that actually computes the rolling mean"""
        if center_b == False:
            dslice_df_ser = data_df_ser[
                ts-pd.datetools.to_offset(window_i_s).delta+timedelta(0,0,1):
                ts
            ]
            # adding a microsecond because when slicing with labels start and endpoint
            # are inclusive
        else:
            dslice_df_ser = data_df_ser[
                ts-pd.datetools.to_offset(window_i_s).delta/2+timedelta(0,0,1):
                ts+pd.datetools.to_offset(window_i_s).delta/2
            ]
        if  (isinstance(dslice_df_ser, pd.DataFrame) and dslice_df_ser.shape[0] < min_periods_i) or \
            (isinstance(dslice_df_ser, pd.Series) and dslice_df_ser.size < min_periods_i):
            return dslice_df_ser.count()*np.nan   # keeps number format and whether Series or DataFrame
        else:
            return dslice_df_ser.count()

    idx_ser = pd.Series(data_df_ser.index.to_pydatetime(), index=data_df_ser.index)
    count_df_ser = idx_ser.apply(calculate_count_ts)
    return count_df_ser


def time_offset_rolling_df_sum(data_df_ser, window_i_s, min_periods_i=1, center_b=False):
    def calculate_sum_ts(ts):
        """Function (closure) to apply that actually computes the rolling mean"""
        if center_b == False:
            dslice_df_ser = data_df_ser[
                ts-pd.datetools.to_offset(window_i_s).delta+timedelta(0,0,1):
                ts
            ]
            # adding a microsecond because when slicing with labels start and endpoint
            # are inclusive
        else:
            dslice_df_ser = data_df_ser[
                ts-pd.datetools.to_offset(window_i_s).delta/2+timedelta(0,0,1):
                ts+pd.datetools.to_offset(window_i_s).delta/2
            ]
        if  (isinstance(dslice_df_ser, pd.DataFrame) and dslice_df_ser.shape[0] < min_periods_i) or \
            (isinstance(dslice_df_ser, pd.Series) and dslice_df_ser.size < min_periods_i):
            return dslice_df_ser.sum()*np.nan   # keeps number format and whether Series or DataFrame
        else:
            return dslice_df_ser.sum()

    idx_ser = pd.Series(data_df_ser.index.to_pydatetime(), index=data_df_ser.index)
    sum_df_ser = idx_ser.apply(calculate_sum_ts)
    return sum_df_ser

In [15]:
last_x_min = "120min"

In [21]:
def df_user_completed(user_id):
    df_user = df_new[(df_new["user_id"] == user_id)].copy()
    df_user = df_user.reset_index()
    df_user = df_user.set_index("ts_datetime")
    df_user = df_user.sort_index()
    df_rolling_count = time_offset_rolling_df_count(df_user, window_i_s=last_x_min)
    df_rolling_sum = time_offset_rolling_df_sum(df_user, window_i_s=last_x_min)

    df_user["count_listened_{}".format(last_x_min)] = df_rolling_count["is_listened"]
    df_user["sum_listened_{}".format(last_x_min)] = df_rolling_sum["is_listened"]
    df_user = df_user.reset_index()
    return df_user

In [22]:
from dask import compute, delayed
values = [delayed(df_user_completed)(user_id) for user_id in users]

In [ ]:
import dask.multiprocessing
results = compute(*values, get=dask.multiprocessing.get)